## Loading Dataset

In [1]:
import numpy as np

In [2]:
X_fn = "./data/combine_astii_astiii_filter_all_smoothing_ipm_E_cloacae_norm.npy"
y_fn = "./data/y_astii_astiii_filter_all_ipm_E_cloacae.npy"
patient_fn = "./data/patient_astii_astiii_filter_all_ipm_E_cloacae.npy"
X_train_raw = np.load(X_fn,allow_pickle=True)
y_train_raw = np.load(y_fn,allow_pickle=True)
patient_train_raw = np.load(patient_fn,allow_pickle=True)

classnames=['CR E_cloacae','CS E_cloacae']



In [3]:
if not isinstance(X_train_raw, np.ndarray):
    X_train_raw = np.array(X_train_raw, dtype=np.float32)
elif X_train_raw.dtype != np.float32:
    X_train_raw = X_train_raw.astype(np.float32)

## Model Setting

In [ ]:
# model
from Variant_LeNet import Variant_LeNet
from tqdm import tqdm
import torch
import pandas as pd
import torch.nn as nn
from torch.autograd import Variable
from FocalLoss import FocalLoss
from losses import AdaptiveProxyAnchorLoss
from imblearn.metrics import specificity_score
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    auc,
    roc_curve,
)
from pytorchtools import EarlyStopping
# Plot
from plot import plot_E_cloacae_CS_E_cloacae_ROC_curve, plot_heatmap,plot_tsne_interactive_html
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn.metrics import confusion_matrix
from config import ORDER, STRAINS
import math

# Training setting
n_classes = 2
epochs = 300 
batch_size = 256
num = 10
num_workers = 7

# metrics
train_avg_accuracy = []
avg_accuracy = []
avg_roc = []
C_total = np.zeros((n_classes, n_classes))
C = np.zeros((n_classes, n_classes))
C_new = np.zeros((n_classes, n_classes))

2025-07-18 09:58:01.078682: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-18 09:58:01.078710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-18 09:58:01.079481: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
class Solver:
    def __init__(
        self,
        i,
        dataloaders,
        model,
        model_path,
        device,
        n_classes,
        alpha,
        gpu=-1,
    ):
        self.i = i
        self.dataloaders = dataloaders
        self.device = device
        self.net = model
        self.net = self.net.to(self.device)
        self.n_classes = n_classes
        self.criterion_proxy =AdaptiveProxyAnchorLoss(nb_classes=n_classes, sz_embed=256, mrg=0.5, alpha=32,\
                                             nb_proxies=1, scale_margin=1).cuda()
        self.criterion = FocalLoss(alpha=alpha, gamma=2).cuda()
        self.get_num_labeled_class = 2
        self.param_groups = [
            {'params': list(set(model.parameters()).difference(set(model.embedding.parameters())))},
            {'params': model.embedding.parameters() , 'lr':float(0.001) * 1},
            {'params': self.criterion_proxy.mrg,'lr':float(0.001) },
            {'params': self.criterion_proxy.proxies, 'lr':float(0.001) * 100}
        ]
        self.optimizer =torch.optim.AdamW(self.param_groups, lr=0.001, weight_decay = 0.001)
        self.model_path = model_path


    def iterate(self, epoch, phase):
        if phase == "train":
            self.net.train()
        else:
            self.net.eval()

            
        dataloader = self.dataloaders[phase]
        total_loss = 0
        correct = 0
        total = 0
        total_proxy_loss = 0
        total_crossentropy_loss = 0
            
        for batch_idx, (inputs, targets,patients) in enumerate(dataloader):
            inputs, targets,patients = Variable(inputs).to(self.device), Variable(targets.long()).to(self.device), patients
            #print(inputs.shape)
            out,emb= self.net(inputs)
            
            loss1 = self.criterion(out, targets)
            loss2 = self.criterion_proxy(emb, targets)
            loss = loss1 + 0.5 * loss2
            outputs = torch.argmax(out.detach(), dim=1)
            
            if phase == "train":
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                torch.nn.utils.clip_grad_value_(self.net.parameters(), 10)
                torch.nn.utils.clip_grad_value_(self.criterion_proxy.parameters(), 10)


            total_loss += loss.item()
            total_proxy_loss += loss2.item()
            total_crossentropy_loss += loss1.item()
            total += targets.size(0)
            correct += outputs.eq(targets).cpu().sum().item()

        total_loss /= (batch_idx + 1)
        total_acc = correct / total

                
        print("\nepoch %d: %s average loss: %.3f | acc: %.2f%% (%d/%d)"
                % (epoch + 1, phase, total_loss, 100.0 * total_acc, correct, total))

        return total_loss,total_acc,out,targets
            

    def train(self, epochs):
        best_loss = float("inf")
        best_acc = 0
        epoch_breaks_classifer = 0
        train_losses = []
        train_acces = [] 
        test_losses = []
        test_acces = [] 

        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(self.optimizer, T_max=60)

        early_stopping_classifier = EarlyStopping(patience=25, mode='acc', verbose=True)

        for epoch in tqdm(range(epochs)):
            train_loss,train_acc,out,targets =self.iterate(epoch, "train")

            train_losses.append(train_loss)
            train_acces.append(train_acc)

            with torch.no_grad():
                
                test_loss,test_acc,out,targets= self.iterate(epoch, "test")
                    
                test_losses.append(test_loss)
                test_acces.append(test_acc)

                if test_acc > best_acc:
                    best_acc = test_acc
                    checkpoint = {
                        "epoch": epoch,
                         "train_acc":train_acc,
                        "test_acc":test_acc,
                        "test_loss": test_loss,
                        "state_dict": self.net.state_dict(),
                    }
                    print("best test acc found")
                    torch.save(checkpoint, self.model_path)
                
                self.scheduler.step()

                early_stopping_classifier(test_acc)

                epoch_breaks_classifer+= 1

                if early_stopping_classifier.early_stop:
                    break  # 停止訓練
        
        if not early_stopping_classifier.early_stop:
            epoch_breaks_classifer = epochs
        

    def test_iterate(self, phase):
        self.net.eval()
        dataloader = self.dataloaders[phase]
        y_pred = []
        y_true = []
        y_pred_prob = []
        
        features_out = []
        features_combine = []
        targets_combine = []
        patient_ids = []
        
        with torch.no_grad():
            for batch_idx, (inputs, targets,patients) in enumerate(dataloader):
                inputs, targets ,patients = Variable(inputs).to('cuda'), Variable(targets.long()).to('cuda'), patients
                out,emb = self.net(inputs)
                outputs = torch.argmax(out.detach(), dim=1)
                outputs_prob = nn.functional.softmax(out.detach(), dim=1)
                y_pred.append(outputs.cpu().numpy())
                y_pred_prob.append(outputs_prob.cpu().numpy())
            
                y_true.append(targets.cpu().numpy())
                
                targets_combine.append(targets.detach().cpu().numpy())
                patient_ids.append(patients)
                features_out.append(out.cpu().numpy())
                features_combine.append(emb.detach().cpu().numpy())

            targets_combine = np.concatenate(targets_combine, axis=0) 
            patient_ids = np.concatenate( patient_ids, axis=0) 
            features_out = np.concatenate(features_out, axis=0)
            features_combine = np.concatenate(features_combine, axis=0)
        
        return (
                np.hstack(y_pred),
                np.hstack(y_true),
                np.vstack(y_pred_prob),
                features_out,
                features_combine,
                targets_combine,
                patient_ids,
            )
    def test(self):
        checkpoint = torch.load(self.model_path)
        epoch = checkpoint["epoch"]
        train_acc = checkpoint["train_acc"]
        test_acc = checkpoint["test_acc"]
        self.net.load_state_dict(checkpoint["state_dict"])
        print("load model at epoch {}, with test acc : {:.3f}".format(epoch+1, test_acc))

        
        _, _ ,_,train_out,train_combined,train_targets,train_patient_ids = self.test_iterate("train")
        y_pred, y_true ,y_pred_prob,out,combined,targets, test_patient_ids,= self.test_iterate("test")
        
        print("total", accuracy_score(y_true, y_pred))
        for i in range(self.n_classes):
            idx = y_true == i
            print("class", i, accuracy_score(y_true[idx], y_pred[idx]))

        return (
            confusion_matrix(y_true, y_pred),
            y_true,
            y_pred,
            accuracy_score(y_true, y_pred),
            y_pred_prob,
            train_acc,
            train_targets,
            targets,
            train_combined,
            combined,
            train_patient_ids,
            test_patient_ids
        )

In [6]:

from datasets_spectrum import spectral_dataloader
from sklearn.model_selection import StratifiedKFold
best_test_accuracy = 0
low_test_accuracy = 1

sss = StratifiedKFold(n_splits=10, shuffle=True,random_state=1)
for fold, (train_index, test_index) in enumerate(sss.split(X_train_raw, y_train_raw)):
    X_train, X_test = X_train_raw[train_index], X_train_raw[test_index]
    y_train, y_test = y_train_raw[train_index], y_train_raw[test_index]
    patient_train ,patient_test = patient_train_raw[train_index],patient_train_raw[test_index]

    dl_tr = spectral_dataloader(
                        X_train, y_train,patient_train, idxs=None, batch_size=batch_size, shuffle=True
                    )
    dl_test = spectral_dataloader(X_test, y_test,patient_test,idxs=None, batch_size=batch_size, shuffle=False)
    values, counts = np.unique(np.asarray(y_test), return_counts=True)
    dataloaders = {"train": dl_tr, "test": dl_test}

    model = Variant_LeNet(in_channels=1, out_channels=n_classes)

    model_path = f"best_variant_lenet_model_{fold}.pth"

    class_counts = np.bincount(y_train)
    num_classes = len(class_counts)
    total_samples = len(y_train)

    class_weights = []
    for count in class_counts:
        weight = 1 / (count / total_samples)
        class_weights.append(weight)
    class_weights = torch.FloatTensor(class_weights)
    alpha = class_weights
    solver = Solver(
                fold,dataloaders, model, model_path, 'cuda', n_classes, alpha
            )
    print(fold + 1)
    solver.train(epochs)
    C, y_true, y_pred, test_accuracy , y_pred_prob,train_acc,train_targets,targets,train_combined,combined ,train_patient_ids, test_patient_ids= solver.test()
    C_total += C  # 將每次迭代的 C 加總到 C_total
    train_avg_accuracy.append(train_acc)
    avg_accuracy.append(test_accuracy)

    if test_accuracy > best_test_accuracy:

        best_test_accuracy = test_accuracy
            
        plot_tsne_interactive_html(f"variant_lenet_best_test_accuracy_combined_train",f"variant_lenet_best_test_accuracy_combined_test",train_combined,combined,train_targets,targets, train_patient_ids, test_patient_ids,classnames )
      
        plot_heatmap(f"variant_lenet_best_test_accuracy_heatmap", C,class_names=classnames)
        
    if test_accuracy < low_test_accuracy:

        low_test_accuracy = test_accuracy
        
        plot_tsne_interactive_html(f"variant_lenet_low_test_accuracy_combined_train",f"variant_lenet_low_test_accuracy_combined_test",train_combined,combined,train_targets,targets,train_patient_ids, test_patient_ids, classnames )
            
        plot_heatmap(f"variant_lenet_low_test_accuracy_heatmap", C,class_names=classnames)
        
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(np.unique(y_true).shape[0]):
        fpr[i], tpr[i], _ = roc_curve(y_test == i, y_pred_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    values = [
            v
            for v in roc_auc.values()
            if isinstance(v, (int, float)) and not math.isnan(v)
    ]
    if values:
        auc_score = sum(values) / len(values)
    avg_roc.append(auc_score)

    for i in range(n_classes):
        C_new[i] = np.round((C[i] / (counts[i] * num)), 2)

        # Plot confusion matrix
    sns.set_context("talk", rc={"font": "Helvetica", "font.size": 12})
    label = [STRAINS[i] for i in ORDER]
    cm = 100 * C_new / C_new.sum(axis=1)[:, np.newaxis]

        # calculate comfusion matrix
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred, average="micro", zero_division=0)
    specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
    f1 = f1_score(y_true, y_pred, average="micro", zero_division=0)

    # metrices result
    df = pd.DataFrame(
        {
            "Accuracy": [np.round(accuracy_score(y_true, y_pred), 4)],
            "Recall": [
                    recall_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "Specificity": [specificity_score(y_true, y_pred, average=None).round(4)],
            "Precision": [
                    precision_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            "F1 Score": [
                    f1_score(y_true, y_pred, average=None, zero_division=0).round(4)
                ],
            }
    )
    print(df.transpose())

    plot_E_cloacae_CS_E_cloacae_ROC_curve(f"variant_lenet_{fold}_roc_curve", y_true, y_test, y_pred_prob)

    plot_heatmap(f"variant_lenet_{fold}_heatmap", cm,class_names=classnames)

plot_heatmap(f"variant_lenet_average_heatmap", C_total,class_names=classnames)


1


  0%|          | 0/300 [00:00<?, ?it/s]


epoch 1: train average loss: 33.188 | acc: 62.78% (1142/1819)


  0%|          | 1/300 [00:00<03:20,  1.49it/s]


epoch 1: test average loss: 29.011 | acc: 13.30% (27/203)
best test acc found

epoch 2: train average loss: 28.035 | acc: 69.16% (1258/1819)


...


epoch 43: test average loss: 35.902 | acc: 95.54% (193/202)
EarlyStopping counter: 24/25 (best: 0.9901)

epoch 44: train average loss: 1.679 | acc: 95.55% (1739/1820)


 14%|█▍        | 43/300 [00:17<01:41,  2.52it/s]


epoch 44: test average loss: 36.249 | acc: 95.54% (193/202)
EarlyStopping counter: 25/25 (best: 0.9901)
🔴 Early stopping triggered
load model at epoch 19, with test acc : 0.990



/tmp/ipykernel_3337346/1054042983.py:175: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



total 0.9900990099009901
class 0 0.9629629629629629
class 1 0.9942857142857143
                           0
Accuracy              0.9901
Recall       [0.963, 0.9943]
Specificity  [0.9943, 0.963]
Precision    [0.963, 0.9943]
F1 Score     [0.963, 0.9943]


In [ ]:
print("max test acc:", np.max(avg_accuracy))
print("min test acc:", np.min(avg_accuracy))

print("train mean:", np.mean(train_avg_accuracy))
print("train std:", np.std(train_avg_accuracy))

    
print("mean:", np.mean(avg_accuracy))
print("std:", np.std(avg_accuracy))

    
print("auc mean:", np.mean(avg_roc))
print("auc std:", np.std(avg_roc))
    

max test acc: 0.9950738916256158
min test acc: 0.9752475247524752
test acc: [0.9901477832512315, 0.9950738916256158, 0.9900990099009901, 0.995049504950495, 0.9801980198019802, 0.9851485148514851, 0.9752475247524752, 0.9851485148514851, 0.9801980198019802, 0.9900990099009901]
test auc: [0.984006734006734, 0.994949494949495, 0.9980332167832167, 0.9991258741258742, 0.9923513986013985, 0.9886363636363638, 0.9980952380952381, 0.9663492063492063, 0.9993650793650795, 0.9938624338624338]
train mean: 0.9309830603700862
train std: 0.01774047742863421
mean: 0.9866409793688729
std: 0.006287448608987739
auc mean: 0.9914775039775041
auc std: 0.009598465056637907
